In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from functools import reduce

from src.data import make_dataset
from src.features import  build_features

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s

In [3]:
df_nacional = pl.scan_csv("../data/processed/egresos_procesados.csv", infer_schema_length=10000,
                          null_values="Extranjero")

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

Una vez calculadas las metricas, se quiere obtener la posicion de cada hospital para cada diagnostico
en cada año. Las posiciones seran calculadas para distintos estratos de los hospitales analizados
(Ejemplo: Hospitales Públicos, Hospitales Privados, Hospitales en GRD).

## Análisis de Estratos

In [5]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_DEL_TORAX)


In [6]:
cie = build_features.leer_cie().with_columns(
    pl.col("Código").alias("DIAG1")
)

In [14]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

ORDEN_COLS = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "Capítulo",
    "Sección",
    "Categoría",
    "Descripción",
    "DIAG1",
    "n_egresos",
    "dias_estada_totales",
    "n_int_q",
    "n_muertos",
    "ranking_nacionales_n_egresos",
    "total_nacionales_n_egresos",
    "%_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
    "total_publicos_n_egresos",
    "%_publicos_n_egresos",
    "ranking_privados_n_egresos",
    "total_privados_n_egresos",
    "%_privados_n_egresos",
    "ranking_grd_n_egresos",
    "total_grd_n_egresos",
    "%_grd_n_egresos",
    "ranking_interno_n_egresos",
    "total_interno_n_egresos",
    "%_interno_n_egresos",
]

In [8]:
dict_resultados_estratos = build_features.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

ranking_nacional_sobre_14 = reduce(build_features.left_join_consecutivo, dict_resultados_estratos.values())
ranking_nacional_con_cie_sobre_14 = ranking_nacional_sobre_14.join(cie, how="left", on="DIAG1").select(ORDEN_COLS)

ranking_nacional_con_cie_pandas_sobre_14 = ranking_nacional_con_cie_sobre_14.to_pandas()
ranking_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14.query("ESTABLECIMIENTO_SALUD == @HOSPITAL_DEL_TORAX")
DIAGS_TORAX = ranking_torax_sobre_14["DIAG1"].unique()

ranking_nacional_diags_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14[ranking_nacional_con_cie_pandas_sobre_14["DIAG1"].isin(DIAGS_TORAX)].query("ANO_EGRESO > 2002")


In [10]:
# ranking_nacional_con_cie_pandas.to_csv(
#     "../data/interim/ranking_nacional_egresos.csv",
#     sep=";",
#     decimal=".",
#     encoding="latin-1",
#     index=False
# )

# ranking_torax.to_csv(
#     "../data/interim/ranking_torax_egresos.csv",
#     sep=";",
#     decimal=".",
#     encoding="latin-1",
#     index=False
# )

# ranking_nacional_diags_torax.to_excel(
#     "../data/interim/ranking_nacional_diags_torax_post_2001.xlsx",
#     index=False
# )



## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.

In [11]:
df_nacional_mayores_a_14 = df_nacional.filter(
    pl.col("EDAD_A_OS") > 14
)

In [12]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(df_nacional_mayores_a_14, AGRUPACION).collect()

In [15]:
dict_resultados_sobre_14 = build_features.obtener_resumen_por_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

ranking_nacional_sobre_14 = reduce(build_features.left_join_consecutivo, dict_resultados_estratos.values())
ranking_nacional_con_cie_sobre_14 = ranking_nacional_sobre_14.join(cie, how="left", on="DIAG1").select(ORDEN_COLS)

ranking_nacional_con_cie_pandas_sobre_14 = ranking_nacional_con_cie_sobre_14.to_pandas()
ranking_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14.query("ESTABLECIMIENTO_SALUD == @HOSPITAL_DEL_TORAX")
DIAGS_TORAX = ranking_torax_sobre_14["DIAG1"].unique()

ranking_nacional_diags_torax_sobre_14 = ranking_nacional_con_cie_pandas_sobre_14[ranking_nacional_con_cie_pandas_sobre_14["DIAG1"].isin(DIAGS_TORAX)].query("ANO_EGRESO > 2002")

In [21]:
ranking_nacional_diags_torax_sobre_14.to_csv("../data/interim/ranking_nacional_sobre_14.csv", index=False, encoding="latin-1", sep=";", decimal=".")

In [26]:
requerimiento = ranking_nacional_diags_torax_sobre_14[ranking_nacional_diags_torax_sobre_14["DIAG1"].isin(["Q230", "Q211", "Q231"])]
requerimiento = requerimiento.query("ANO_EGRESO == 2019")

In [27]:
requerimiento.to_csv("../data/interim/ranking_nacional_sobre_14.csv", index=False, encoding="latin-1", sep=";", decimal=".")

In [28]:
requerimiento

,ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,...,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
225208,2019.0,111200.0,Hospital Clínico Universidad Católica,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q23 MALFORMACIONES CONGÉNITAS DE LAS VÁLVULAS ...,Insuficiencia congénita de la válvula aórtica,Q231,7,88,...,NaN,1.0,18.0,0.388889,NaN,NaN,NaN,NaN,NaN,NaN
225209,2019.0,112200.0,Clínica Alemana,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q23 MALFORMACIONES CONGÉNITAS DE LAS VÁLVULAS ...,Insuficiencia congénita de la válvula aórtica,Q231,6,36,...,NaN,2.0,18.0,0.333333,NaN,NaN,NaN,NaN,NaN,NaN
225210,2019.0,112102.0,Hospital de Niños Dr. Luis Calvo Mackenna (San...,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q23 MALFORMACIONES CONGÉNITAS DE LAS VÁLVULAS ...,Insuficiencia congénita de la válvula aórtica,Q231,3,6,...,0.272727,NaN,NaN,NaN,1.0,11.0,0.272727,NaN,NaN,NaN
225211,2019.0,112103.0,Instituto Nacional de Enfermedades Respiratori...,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q23 MALFORMACIONES CONGÉNITAS DE LAS VÁLVULAS ...,Insuficiencia congénita de la válvula aórtica,Q231,2,18,...,0.181818,3.0,18.0,0.111111,2.0,11.0,0.181818,220.0,5435.0,0.000368
225212,2019.0,109101.0,Hospital Clínico de Niños Dr. Roberto del Río ...,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q23 MALFORMACIONES CONGÉNITAS DE LAS VÁLVULAS ...,Insuficiencia congénita de la válvula aórtica,Q231,2,10,...,0.181818,NaN,NaN,NaN,3.0,11.0,0.181818,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225402,2019.0,113190.0,"Hospital Parroquial (Santiago, San Bernardo) (D)","Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q21 MALFORMACIONES CONGÉNITAS DE LOS TABIQUES ...,Defecto del tabique auricular,Q211,1,2,...,0.004545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225403,2019.0,110100.0,"Hospital San Juan de Dios (Santiago, Santiago)","Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q21 MALFORMACIONES CONGÉNITAS DE LOS TABIQUES ...,Defecto del tabique auricular,Q211,1,1,...,0.004545,NaN,NaN,NaN,21.0,218.0,0.004587,NaN,NaN,NaN
225404,2019.0,111101.0,Hospital Clínico Metropolitano El Carmen Docto...,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q21 MALFORMACIONES CONGÉNITAS DE LOS TABIQUES ...,Defecto del tabique auricular,Q211,1,14,...,0.004545,NaN,NaN,NaN,22.0,218.0,0.004587,NaN,NaN,NaN
225405,2019.0,119102.0,Hospital Penco - Lirquén,"Cap.17 MALFORMACIONES CONGÉNITAS, DEFORMIDADE...",Q20-Q28 MALFORMACIONES CONGÉNITAS DEL APARATO...,Q21 MALFORMACIONES CONGÉNITAS DE LOS TABIQUES ...,Defecto del tabique auricular,Q211,1,14,...,0.004545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
